In [1]:
%load_ext autoreload
%autoreload 2

from typing import Dict, List, Iterator, Union, Any
import logging
import tempfile
import json
import shutil
from pathlib import Path
from pprint import pprint
from models.utils.wandb import RunData
from models.utils.allennlp import (
    load_config, load_dataset_reader, load_iterator, load_model,load_best_metrics, load_modules, 
    load_outputs, create_onepass_generator)
from allennlp.models import Model
from allennlp.data import DatasetReader, DataIterator
import pandas as pd
from copy import deepcopy
import plotly.graph_objs as go
import numpy as np
from ipywidgets import interact, fixed
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s", level=logging.INFO)

logger = logging.getLogger(__name__)


In [12]:
# config
RUN = 'ezmblr2o'
EXTRA_MODULES = ['models', 'datasets']
DATA_DIR = '/Users/dhruv/UnsyncedDocuments/IESL/kb_completion/models/.data'
DOWNLOADS_DIR = '.models/AUG_NEW'
#DOWNLOADS_DIR = '/var/folders/r2/8mbb22091rb4vq4xtd19bbw80000gn/T/tmp5ypyp_4a'

In [13]:
# get the run files
logger.info("Downloading run files if needed...")
rd = RunData(RUN, group='iesl-boxes', project='kb-completion',
             download_dir=Path(DOWNLOADS_DIR))

rd.download_files(skip_patterns=['allennlp_serialization_dir/metrics_epoch_[\d]+.json$'])

2020-02-13 23:58:40,756 - INFO - __main__ - Downloading run files if needed...
2020-02-13 23:58:41,401 - INFO - models.utils.wandb - Setting up download dir as .models/AUG_NEW/ezmblr2o
2020-02-13 23:58:41,652 - INFO - models.utils.wandb - Downloading allennlp_serialization_dir/best.th
2020-02-13 23:58:43,211 - INFO - models.utils.wandb - Downloading allennlp_serialization_dir/config.json
2020-02-13 23:58:43,741 - INFO - models.utils.wandb - Downloading allennlp_serialization_dir/log/train/events.out.tfevents.1581630853.node139
2020-02-13 23:58:48,572 - INFO - models.utils.wandb - Downloading allennlp_serialization_dir/metrics.json
2020-02-13 23:58:57,932 - INFO - models.utils.wandb - Downloading allennlp_serialization_dir/model.tar.gz
2020-02-13 23:58:59,667 - INFO - models.utils.wandb - Downloading allennlp_serialization_dir/model_state_epoch_1848.th
2020-02-13 23:59:01,503 - INFO - models.utils.wandb - Downloading allennlp_serialization_dir/stderr.log
2020-02-13 23:59:02,259 - INFO -

In [14]:

# load modules
logger.info("Loading {} for AllenNLP".format(EXTRA_MODULES))
load_modules(EXTRA_MODULES)

# override the val dataset setup to become the test dataset setup
overrides = {'validation_dataset_reader':
             {
                 'all_datadir': DATA_DIR,
                 "validation_file": "classification_samples_test2id.txt"
             }
            }
logger.info("Setting up config overrides ...")

# load config
config = load_config(
    rd.download_dir/"allennlp_serialization_dir", overrides_dict=overrides)
#pprint(config.as_dict())

# load best metrics
best_metrics = load_best_metrics(rd.download_dir/"allennlp_serialization_dir")
# load model
model = load_model(rd.download_dir/"allennlp_serialization_dir", config=config)
model.test_threshold = best_metrics['best_validation_threshold']

# create dataset reader
dataset_reader = load_dataset_reader(config=config)

# create iterator
# This will take time for the first time as it will create the cache
iterator = load_iterator(config=config)
generator = create_onepass_generator(iterator, dataset_reader)

2020-02-13 23:59:10,328 - INFO - __main__ - Loading ['models', 'datasets'] for AllenNLP
2020-02-13 23:59:10,427 - INFO - __main__ - Setting up config overrides ...
2020-02-13 23:59:10,485 - INFO - allennlp.common.params - Converting Params object to dict; logging of default values will not occur when dictionary parameters are used subsequently.
2020-02-13 23:59:10,486 - INFO - allennlp.common.params - CURRENTLY DEFINED PARAMETERS: 
2020-02-13 23:59:10,487 - INFO - allennlp.common.params - train_data_path = dummpy_path
2020-02-13 23:59:10,496 - INFO - allennlp.common.params - validation_dataset_reader.all_datadir = /mnt/nfs/work1/mccallum/dhruveshpate/kb_completion/models/.data
2020-02-13 23:59:10,497 - INFO - allennlp.common.params - validation_dataset_reader.dataset_name = AUG_NEW
2020-02-13 23:59:10,502 - INFO - allennlp.common.params - validation_dataset_reader.type = classification-validation-dataset
2020-02-13 23:59:10,503 - INFO - allennlp.common.params - validation_dataset_reade

2020-02-13 23:59:10,770 - INFO - allennlp.common.params - validation_dataset_reader.dataset_name = AUG_NEW
2020-02-13 23:59:10,771 - INFO - allennlp.common.params - validation_dataset_reader.all_datadir = /Users/dhruv/UnsyncedDocuments/IESL/kb_completion/models/.data
2020-02-13 23:59:10,774 - INFO - allennlp.common.params - validation_dataset_reader.validation_file = classification_samples_test2id.txt
2020-02-13 23:59:10,776 - INFO - allennlp.common.from_params - instantiating class <class 'allennlp.data.iterators.data_iterator.DataIterator'> from params {'batch_size': 5000, 'cache_instances': True, 'type': 'basic'} and extras set()
2020-02-13 23:59:10,779 - INFO - allennlp.common.params - validation_iterator.type = basic
2020-02-13 23:59:10,781 - INFO - allennlp.common.from_params - instantiating class <class 'allennlp.data.iterators.basic_iterator.BasicIterator'> from params {'batch_size': 5000, 'cache_instances': True} and extras set()
2020-02-13 23:59:10,784 - INFO - allennlp.commo

In [15]:
from typing import Iterator, Union, Any
import torch, tqdm
from allennlp.nn import util as nn_util

def predict_loop_or_load(
        model: Model,
        dataset_iterator: Iterator,
        device: str = 'cpu',
        output_file: Union[str, Path] = 'output.jsonl',
        force_repredict: bool = False) -> List[Dict[str, Any]]:
    """
    Checks if results are already present in the output file. If the file exists reads it and returns
    the contents. If it does not, runs the prediction loop and populates the file and returns results.
    """
    # check
    output_file: Path = Path(output_file)  # type: ignore

    if output_file.exists():
        if output_file.is_file():
            logger.info("{} file already exists...")

            if force_repredict:
                logger.info(
                    "force_repredict is True. Hence repredicting and overwritting"
                )
            else:
                logger.info("Reading results from the existing file ")

                return load_outputs(output_file)

    # Predict
    device = 'cpu'

    if device is 'cpu':
        device_instance = torch.device('cpu')
        device_int = -1
    else:
        device_instance = torch.device('cuda', 0)
        device_int = 0
    model = model.to(device=device_instance)
    model.eval()
    model.test()
    with open(output_file, 'w') as f:
        logger.info("Starting predictions ...")

        for i, input_batch in enumerate(tqdm.tqdm(dataset_iterator)):
            input_batch_on_device = nn_util.move_to_device(
                input_batch, device_int)
            result = model.forward(**input_batch_on_device)
            print(model.get_metrics())
        logger.info('Prediction complete')
        metrics = model.get_metrics()
        logger.info("Test metrics\n")
        print(metrics)
        json.dump(metrics, f)
    return metrics

In [16]:
# Predict
results = predict_loop_or_load(model, generator, device='cpu', output_file=Path(rd.download_dir)/'test_outputs.jsonl', force_repredict=True)

2020-02-13 23:59:31,033 - INFO - __main__ - Starting predictions ...
1it [00:00,  2.86it/s]

{'precision': 1.0, 'recall': 0.6291999816894531, 'fscore': 0.7724035978317261}


2it [00:00,  2.93it/s]

{'precision': 1.0, 'recall': 0.6290000081062317, 'fscore': 0.7722529768943787}


3it [00:00,  3.00it/s]

...

{'precision': 0.7338330745697021, 'recall': 0.6309027671813965, 'fscore': 0.6784863471984863}


209it [01:21,  2.96it/s]

{'precision': 0.7332932949066162, 'recall': 0.6309027671813965, 'fscore': 0.678255558013916}


209it [01:23,  2.51it/s]
2020-02-14 00:00:54,438 - INFO - __main__ - Prediction complete
2020-02-14 00:00:54,441 - INFO - __main__ - Test metrics



{'precision': 0.7332932949066162, 'recall': 0.6309027671813965, 'fscore': 0.678255558013916}
